In [1]:
# Import libraries
import gym, retro # For the environment
import time  # For slowing down fights
import os
import sys

# Import the libraries necessary for data preprocessing.

from gym import Env  # Base environment class for a wrapper
from gym.spaces import MultiBinary, Box  # Ensure we pick the correct action space type. (Space shapes for the environment)
import numpy as np  # To calculate frame delta
import cv2  # For grayscaling
from matplotlib import pyplot as plt  # For plotting observation images

import optuna  # Importing the optimization framework that allows to both train and tune at the same time
import torch
import os  # For exporting the model
from stable_baselines3 import PPO  # PPO algorithm for RL
from stable_baselines3.common.evaluation import evaluate_policy  # Metric calculation of agent performance
from stable_baselines3.common.monitor import Monitor  # SB3 Monitor for logging
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack  # Vec wrappers to vectorize and frame stack
import tensorboard as tb

c:\Users\diego\Desktop\street_fighter\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Class definition
class StreetFighter(Env):
    def __init__(self):

        # Inherit from our base environment
        super().__init__()

        # Specify action and observation spaces
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)  # We create our observation space based on the new size and colors
        self.action_space = MultiBinary(12)  # We replicate the base action environment

        # Startup and instance the game
        # The second parameter will limit actions to only valid ones.
        try:
            self.game = retro.make(
                game='StreetFighterIISpecialChampionEdition-Genesis', 
                use_restricted_actions=retro.Actions.FILTERED
            )
        except Exception as e:
            print(f"Error creating retro environment: {e}")
            sys.exit(1)


    def reset(self):
        # Return first frame, preprocess the frame, and define score back to 0.

        self.previous_frame = np.zeros(self.game.observation_space.shape)

        obs = self.game.reset()  # Will return our observation
        obs = self.preprocess(obs)  # We preprocess the observation

        self.health = 176  # Initial health
        self.enemy_health = 176

        self.matches_won = 0
        self.enemy_matches_won = 0
        
        
        # Game delta = Current_frame - Previous_frame
        # Preprocess
        self.previous_frame = obs

        # Attribute to hold delta score.
        self.score = 0

        return obs
    
    def preprocess(self, observation):
        # Grayscale, and resize frame
        
        # Grayscaling
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)

        # Resizing
        resize = cv2.resize(gray, (84, 84), interpolation=cv2.INTER_CUBIC)
        
        channel = np.reshape(resize, (84, 84, 1))  # We add the grayscale layer since its what gym expects

        return channel

    def step(self, action):
        # We take a step, preprocess the observation, calculate frame delta and reshape the reward function

        # Take a step
        obs, reward, done, info = self.game.step(action)  # New step based on an action

        obs = self.preprocess(obs)  # We preprocess the observation

        # Frame delta

        # We subtract the current one from the previous one and then we set the current as the last one.
        frame_delta = obs  # - self.previous_frame
        # self.previous_frame = obs

        # Reshape the reward function based on relative score
        # reward = info['score'] - self.score  # Current reward minus the previous score
        # self.score = info['score']  # We set our score to the current score.
        
        delta_enemy = (self.enemy_health - info['enemy_health']) / 176
        delta_self = (info['health'] - self.health) / 176
        reward = delta_enemy * 10 - delta_self * 5

        if abs(delta_enemy) < 1e-6 and abs(delta_self) < 1e-6:
            reward -= 0.001 # small penalty for idling

        if done:
            reward += (info['matches_won'] - info['enemy_matches_won']) * 20  # match win/loss bonus
            if info["enemy_health"] <= 0:
                reward += 50  # big win bonus
            elif info["health"] <= 0:
                reward -= 60  # big loss penalty
            


                
        delta_score = info.get('score', 0) - getattr(self, 'score', 0)
        self.score = info.get('score', self.score)
        reward += delta_score * 0.001


       # Update values
        self.health = info['health']
        self.enemy_health = info['enemy_health'] 

        return frame_delta, reward, done, info


    def render(self, *args, **kwargs):
        # We render the game
        self.game.render()

    def close(self):
        # We close the game
        self.game.close()


In [6]:
# Directories where saved optimization models are going to be saved

LOG_DIR = './logs/'  # SB3 has the ability to log out to a support log
OPT_DIR = './opt/'  # Location to save every single model after every try

# Model definition
# env.close()
env = StreetFighter()
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [33]:
env.close()

In [21]:
env = retro.make(
    game='StreetFighterIISpecialChampionEdition-Genesis', 
    use_restricted_actions=retro.Actions.FILTERED
)

env.observation_space = Box(low=0, high=255, shape=(4, 84, 84), dtype=np.uint8)

In [16]:
model = PPO.load('./train/27500000.zip')

# mean_reward, _ = evaluate_policy(model, env, render=False, n_eval_episodes=5)
# mean_reward

In [17]:
# Test to see everything working

# Reset game to starting state
obs = env.reset()

# Flag to false
done = False

tick = 0

max_ticks = 10000

for game in range(1):

    # If game is not over.
    while not done: # and tick < max_ticks
        if done:
            # We reset the game
            obs = env.reset()

        tick += 1

        # Render environment
        env.render()

        action = model.predict(obs)[0]
        obs, reward, done, info = env.step(action)

        # # We slow down the renders so they are watchable
        # time.sleep(0.01)

        # # We print the reward
        # if reward != 0:
        #     print(reward)
    # obs = env.reset()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [26]:
import numpy as np

# Test to see everything working
obs = env.reset()
done = False
tick = 0
max_ticks = 10000

for game in range(1):
    
    while not done and tick < max_ticks:
        
        tick += 1
        env.render()

        try:
            # Intentar con la observación directamente
            action = model.predict(obs, deterministic=True)[0]
        except Exception as e:
            print(f"Error con observación directa: {e}")
            try:
                # Intentar expandiendo dimensiones
                processed_obs = np.expand_dims(obs, axis=0)
                action = model.predict(processed_obs, deterministic=True)[0]
            except Exception as e2:
                print(f"Error también con dimensión expandida: {e2}")
                # Usar acción por defecto
                action = env.action_space.sample()

        obs, reward, done, info = env.step(action)

        time.sleep(.01)  # Pause before starting a new game

        # Opcional: imprimir información de depuración
        if tick % 100 == 0:
            print(f"Tick: {tick}, Reward: {reward}, Done: {done}")
            print(f"Obs shape: {obs.shape if hasattr(obs, 'shape') else 'dict'}")
    
    if done:
        print("Game finished! Resetting...")
        obs = env.reset()
        done = False
        tick = 0



Error con observación directa: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
Error también con dimensión expandida: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
Error con observación directa: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
Error también con dimensión expandida: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
Error con observación directa: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
Error también con dimensión expandida: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
Error con observación directa: only integers, slices (`:`), ellipsis (`...`), nump

In [27]:
env.close()

In [ ]:
env2 = retro.make(
    game='StreetFighterIISpecialChampionEdition-Genesis', 
    use_restricted_actions=retro.Actions.FILTERED
)

In [ ]:
# Reset game to starting state
obs = env2.reset()

# Flag to false
done = False

# We only play one game
for game in range(1):

    # If game is not over.
    while not done:
        if done:
            # We reset the game
            obs = env2.reset()

        # Render environment
        env2.render()

        # We take random actions inside the environment
        obs, reward, done, info = env2.step(env2.action_space.sample())

        # We slow down the renders so they are watchable
        # time.sleep(0.01)

        # We print the reward
        print(reward)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [12]:
%pip install pygame==1.9.6

Note: you may need to restart the kernel to use updated packages.Collecting pygame==1.9.6



You should consider upgrading via the 'c:\Users\diego\Desktop\street_fighter\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [28]:
try: 
    env = retro.make(
        game='StreetFighterIISpecialChampionEdition-Genesis',
        use_restricted_actions=retro.Actions.FILTERED
    )
except Exception as e:
    try:
        env.close()
        env = retro.make(
        game='StreetFighterIISpecialChampionEdition-Genesis',
        use_restricted_actions=retro.Actions.FILTERED
        )
    except Exception as e:
        print(f"Error creating retro environment: {e}")
        

Exception ignored in: <function SimpleImageViewer.__del__ at 0x00000207055D5EE0>
Traceback (most recent call last):
  File "c:\Users\diego\Desktop\street_fighter\venv\lib\site-packages\gym\envs\classic_control\rendering.py", line 458, in __del__
    self.close()
  File "c:\Users\diego\Desktop\street_fighter\venv\lib\site-packages\gym\envs\classic_control\rendering.py", line 454, in close
    self.window.close()
  File "c:\Users\diego\Desktop\street_fighter\venv\lib\site-packages\pyglet\window\win32\__init__.py", line 330, in close
    super(Win32Window, self).close()
  File "c:\Users\diego\Desktop\street_fighter\venv\lib\site-packages\pyglet\window\__init__.py", line 858, in close
    app.windows.remove(self)
  File "C:\Users\diego\AppData\Local\Programs\Python\Python38\lib\_weakrefset.py", line 109, in remove
    self.data.remove(ref(item))
KeyError: <weakref at 0x00000207031DEC70; to 'Win32Window' at 0x0000020702DED190>


In [ ]:
env.close()
env = retro.make(
    game='StreetFighterIISpecialChampionEdition-Genesis',
    state= "<State.MainMenu>",
    # use_restricted_actions=retro.Actions.FILTERED
)

In [ ]:
# env.close()
# env = retro.RetroEnv(
#     game='StreetFighterIISpecialChampionEdition-Genesis',
#     # state='MainMenu',
#     # use_restricted_actions=retro.Actions.FILTERED
# )

Exception ignored in: <function SimpleImageViewer.__del__ at 0x00000207055D5EE0>
Traceback (most recent call last):
  File "c:\Users\diego\Desktop\street_fighter\venv\lib\site-packages\gym\envs\classic_control\rendering.py", line 458, in __del__
    self.close()
  File "c:\Users\diego\Desktop\street_fighter\venv\lib\site-packages\gym\envs\classic_control\rendering.py", line 454, in close
    self.window.close()
  File "c:\Users\diego\Desktop\street_fighter\venv\lib\site-packages\pyglet\window\win32\__init__.py", line 330, in close
    super(Win32Window, self).close()
  File "c:\Users\diego\Desktop\street_fighter\venv\lib\site-packages\pyglet\window\__init__.py", line 858, in close
    app.windows.remove(self)
  File "C:\Users\diego\AppData\Local\Programs\Python\Python38\lib\_weakrefset.py", line 109, in remove
    self.data.remove(ref(item))
KeyError: <weakref at 0x0000020701331400; to 'Win32Window' at 0x000002070CD0E520>


In [45]:
env.buttons

['B', 'A', 'MODE', 'START', 'UP', 'DOWN', 'LEFT', 'RIGHT', 'C', 'Y', 'X', 'Z']

In [ ]:
import retro
import pygame
import numpy as np

# Initialize pygame
pygame.init()
screen = pygame.display.set_mode((400, 300))
pygame.display.set_caption("Street Fighter II Manual Play")

# Create the environment
try: 
    env = retro.make(
        game='StreetFighterIISpecialChampionEdition-Genesis',
        # state=None
        # use_restricted_actions=retro.Actions.FILTERED
    )
except Exception as e:
    try:
        env.close()
        env = retro.make(
        game='StreetFighterIISpecialChampionEdition-Genesis',
        # state=None
        # use_restricted_actions=retro.Actions.FILTERED
        )
    except Exception as e:
        print(f"Error creating retro environment: {e}")
        pygame.quit()
        exit(1)


obs = env.reset()
done = False

# Define key mapping for actions (matching your button order)
# ['B', 'A', 'MODE', 'START', 'UP', 'DOWN', 'LEFT', 'RIGHT', 'C', 'Y', 'X', 'Z']
key_map = {
    pygame.K_s: 0,      # 'B'
    pygame.K_a: 1,      # 'N'
    pygame.K_m: 2,      # 'MODE'
    pygame.K_SPACE: 3, # 'START'
    pygame.K_UP: 4,     # 'UP'
    pygame.K_DOWN: 5,   # 'DOWN'
    pygame.K_LEFT: 6,   # 'LEFT'
    pygame.K_RIGHT: 7,  # 'RIGHT'
    pygame.K_d: 8,      # 'C'
    pygame.K_z: 9,      # 'Y'
    pygame.K_x: 10,     # 'X'
    pygame.K_c: 11,     # 'Z'
}

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            env.close()
            pygame.quit()
            exit()

    # ✅ Check which keys are currently pressed
    keys = pygame.key.get_pressed()
    action = np.zeros(env.action_space.n, dtype=np.int8)

    for key, index in key_map.items():
        if keys[key]:
            action[index] = 1
            # print(f"Key {pygame.key.name(key)} pressed, setting action index {index} to 1")

    # Step environment with current action
    print(action)
    obs, reward, done, info = env.step(action)

    time.sleep(0.01)  # Optional: slow down the loop for better control
    env.render()

    

    # if done:
    #     obs = env.reset()

Exception ignored in: <function SimpleImageViewer.__del__ at 0x0000024451B03160>
Traceback (most recent call last):
  File "c:\Users\diego\Desktop\street_fighter\venv\lib\site-packages\gym\envs\classic_control\rendering.py", line 458, in __del__
    self.close()
  File "c:\Users\diego\Desktop\street_fighter\venv\lib\site-packages\gym\envs\classic_control\rendering.py", line 454, in close
    self.window.close()
  File "c:\Users\diego\Desktop\street_fighter\venv\lib\site-packages\pyglet\window\win32\__init__.py", line 330, in close
    super(Win32Window, self).close()
  File "c:\Users\diego\Desktop\street_fighter\venv\lib\site-packages\pyglet\window\__init__.py", line 858, in close
    app.windows.remove(self)
  File "C:\Users\diego\AppData\Local\Programs\Python\Python38\lib\_weakrefset.py", line 109, in remove
    self.data.remove(ref(item))
KeyError: <weakref at 0x000002444C754B30; to 'Win32Window' at 0x0000024449E72F70>


[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0

KeyboardInterrupt: 